# Importações

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.io import imread, imsave,imread_collection
from skimage.color import rgb2gray
from scipy import interpolate
from skimage.exposure import histogram
import glob
import joblib 
import time

## Lendo as imagens saudáveis da base rimone-v2

In [3]:
base_normal = imread_collection('bases/rimone-v2/Normal/*.jpg')
print(len(base_normal))

2


## Lendo as imagens doentes da base rimone-v2

In [3]:
base_doente = imread_collection('bases/rimone-v2/Doentes/*.jpg')
print(len(base_doente))

200


### Convertendo as imagens saudáveis de RGB para tons de cinza e salvando na pasta especificas

In [ ]:
#Convertendo as imagens nos canais RGB
for i, img in enumerate(base_normal):
    #canal vermelho
    red = img[:,:,0]
    canal_vermelho =rgb2gray(red)
    imsave('bases/rimone-v2/cinza_normal/cinza_red/im_normal_red'+(str(i))+'.jpg', canal_vermelho)

    #canal verde
    green = img[:,:,1]
    canal_verde =rgb2gray(green)
    imsave('bases/rimone-v2/cinza_normal/cinza_green/im_normal_green'+(str(i))+'.jpg', canal_verde)

    #canal azul
    blue = img[:,:,2]
    canal_azul =rgb2gray(blue)
    imsave('bases/rimone-v2/cinza_normal/cinza_blue/im_normal_blue'+(str(i))+'.jpg', canal_azul)

### Convertendo as imagens doentes de RGB para tons de cinza e salvando na pasta especificas

In [ ]:
#Convertendo as imagens nos canais RGB
for i, img in enumerate(base_doente):
    #canal vermelho
    red = img[:,:,0]
    canal_vermelho =rgb2gray(red)
    imsave('bases/rimone-v2/cinza_doente/cinza_red/im_doente_red'+(str(i))+'.jpg', canal_vermelho)

    #canal verde
    green = img[:,:,1]
    canal_verde =rgb2gray(green)
    imsave('bases/rimone-v2/cinza_doente/cinza_green/im_doente_green'+(str(i))+'.jpg', canal_verde)

    #canal azul
    blue = img[:,:,2]
    canal_azul =rgb2gray(blue)
    imsave('bases/rimone-v2/cinza_doente/cinza_blue/im_doente_blue'+(str(i))+'.jpg', canal_azul)

# Padrões Binários Locais (LBP)

A definição do método de extração de características é um dos fatores mais importantes para a construção de um sistema de visão ou reconhecimento de padrões. O desempenho de um sistema de visão está diretamente relacionado ao poder de discriminação do conjunto de características escolhido. As características devem destacar a diferenças entre objetos de classes distintas (interclasses) e minimizar possíveis diferenças de objetos de mesma classe (intraclasse). 

O objetivo comum da extração de características é representar os dados brutos como um conjunto reduzido de recursos que descrevem melhor suas principais características e atributos. Desta forma, podemos reduzir a dimensionalidade da entrada original e usar os novos recursos como uma entrada para treinar técnicas de reconhecimento e classificação de padrões.

Embora existam vários recursos que podemos extrair de uma imagem, os Padrões Binários Locais (LBP) são uma abordagem teoricamente simples, mas eficiente, para classificação de texturas invariante de escala de cinza e rotação. Eles funcionam porque os padrões mais freqüentes correspondem a microcaracterísticas primitivas, como bordas, cantos, manchas, regiões planas.

A técnica LBP rotula os pixels de uma imagem analisando a intensidade dos pixels de uma vizinhança de 8 com o seu pixel central. É um descritor de textura que extrai informações de variação dos brilhos entre pixels vizinhos. Para isso, é definida uma janela com raio e uma quantidade de vizinhos p.

$ LBP_ {P, R} $ operador é por definição invariante contra qualquer transformação monotônica da escala de cinza. Contanto que a ordem dos valores de cinza permaneça a mesma, a saída do operador $ LBP_ {P, R} $ permanece constante.

$$LBP_{P,R} = \sum_{p = 0} ^ {P-1} s (g_p - g_c) 2 ^ p $$

Onde

$$s(x) = 
\begin{cases} 
    1 & x\geq0 \\
    0 & x<0
\end{cases}
$$

### Conjunto vizinho simétrico circular

Um vizinho circularmente simétrico definido para um determinado pixel $ g_c $ é definido pelos pontos com coordenadas (i, j) que circundam o ponto central em um círculo de raio R e o número de elementos P.

$$i = -R\sin\big(\frac{2\pi p}{ P}\big);\hspace{1em}j = R\cos\big(\frac{2\pi p}{P}\big)\hspace{1em}\forall\hspace{0.5em}p \in \{0, 1, ..., P\}$$

In [4]:
def vizinhos(p_vizinhos, raio):
    x = np.arange(0, p_vizinhos)
    x = raio * np.cos(2 * np.pi * x / p_vizinhos)
    
    y = np.arange(0, p_vizinhos)
    y = - raio * np.sin(2 * np.pi * y / p_vizinhos)
    
    return x, y

## Interpolação
Quando um vizinho não está localizado no centro de um pixel, esse valor cinza vizinho deve ser calculado por interpolação. Assim, precisamos definir uma função que, dada uma coordenada, retorne o valor cinza interpolado.

Pode-se expressar a interpolação por vizinho mais próximo por meio da equação 
sendo f(x, y) o pixel a ser interpolado, em que dx e dy são as distâncias entre os pontos originais e os interpolados nas direções x e y, respectivamente. Ou seja, dx = x − x e dy = y − y.


In [1]:
def interpolacao2d(imagem, kind='cubic'):
    '''
    Retorna uma função f (x, y) que retorna o valor interpolado
    dos pontos (x, y).
    '''
    
    assert imagem.ndim == 2, 'Deve ser uma imagem bidimensional'
    h, w = imagem.shape

    x = np.arange(0, w)
    y = np.arange(0, h)

    return interpolate.interp2d(x, y, imagem, kind=kind)

def calcular_valores_vizinhos(x, y, funcao_interpolacao):
    '''
        A função zip () aceita iteráveis (pode ser zero ou mais), faz iterador que agrega elementos 
        baseados nos iteráveis passados e retorna um iterador de tuplas.
        
        A função map () aplica uma determinada função a cada item de um iterável (lista, tupla, etc.) e 
        retorna uma lista dos resultados.
    '''
    
    valores_cinza = map(lambda pontos: funcao_interpolacao(*pontos), zip(x, y))
    return np.fromiter(valores_cinza, float)

####  Distribuição Binária Conjunta

Considerando uma possível perda de informação, é possível transformar a textura na diferença conjunta. Para calculá-lo, subtraímos o valor cinza do pixel central para todo o conjunto vizinho. A distribuição de diferença conjunta é um operador de textura altamente discriminativa. Ele registra as ocorrências de vários padrões na vizinhança de cada pixel em um histograma P-dimensional.

In [6]:
def distribuicao_binaria_conjunta(gc, x, y, interpolacao_f):
    
    #distribuicao de diferenca conjunta
    xc, yc = gc
    
    xp = xc + x
    yp = yc + y
    
    g_p = calcular_valores_vizinhos(xp, yp, interpolacao_f)
    g_c = interpolacao_f(xc, yc)
    
    diferenca =  np.round(g_p - g_c, 15)
    
    return np.where( diferenca >= 0, 1, 0)

def LBP(gc, x, y, interpolacao_f,p_vizinhos):
    s = distribuicao_binaria_conjunta(gc, x, y, interpolacao_f)
    p = np.arange(0, p_vizinhos)
    fator_binomial = 2 ** p
    return np.sum(fator_binomial * s)

In [7]:
def filtro_LBP(imagem, raio, p_vizinhos):
    x, y = vizinhos(p_vizinhos, raio)
    h, w = imagem.shape
    f = interpolacao2d(imagem,kind='cubic')
    imagem_LBP = np.zeros((h, w),dtype=imagem.dtype)

    for j in range(h):
        for i in range(w):
            imagem_LBP[j, i] = LBP((i, j), x, y, f ,p_vizinhos)
    return imagem_LBP

### Testando o Descritor de Textura LBP

    
   O LBP padrão utiliza raio unitário (raio = 1) de vizinhana 8 (p_vizinhos = 8). 
   A seguir serão realizadas 12 combinações no total. Dentre essas combinações as combinações 2, 6 e 10 utilizam o LBP padrão nos três canais RGB na escala de cinza.

## Lendo a nova base do canal em tons de cinza normal nos tres canais RGB

In [8]:
# RED
base_nomal_red = imread_collection('bases/rimone-v2/cinza_normal/cinza_red/*.jpg')
print(len(base_nomal_red))
# GREEN
base_nomal_green = imread_collection('bases/rimone-v2/cinza_normal/cinza_green/*.jpg')
print(len(base_nomal_green))
# BLUE
base_nomal_blue = imread_collection('bases/rimone-v2/cinza_normal/cinza_blue/*.jpg')
print(len(base_nomal_blue))

255
255
255


## Lendo a nova base do canal em tons de cinza doentes nos tres canais RGB

In [9]:
# RED
base_doente_red = imread_collection('bases/rimone-v2/cinza_doente/cinza_red/*.jpg')
print(len(base_doente_red))
# GREEN
base_doente_green = imread_collection('bases/rimone-v2/cinza_doente/cinza_green/*.jpg')
print(len(base_doente_green))
# BLUE
base_doente_blue = imread_collection('bases/rimone-v2/cinza_doente/cinza_blue/*.jpg')
print(len(base_doente_blue))

200
200
200


## Função Calcula Canal
   * parametros
       * canal
       * raio
       * p_vizinhos

### combinações a serem executas nos tres canais RGB tanto nas imagens saudáveis como nas doentes
   #### RED
   * raio = 1, p_viznho=4, canal=red
   * raio = 2, p_viznho=4, canal=red
   * raio = 1, p_viznho=8, canal=red
   * raio = 2, p_viznho=8, canal=red
   #### GREEN
   * raio = 1, p_viznho=4, canal=green
   * raio = 2, p_viznho=4, canal=green
   * raio = 1, p_viznho=8, canal=green
   * raio = 2, p_viznho=8, canal=green
   #### BLUE   
   * raio = 1, p_viznho=4, canal=blue
   * raio = 2, p_viznho=4, canal=blue
   * raio = 1, p_viznho=8, canal=blue
   * raio = 2, p_viznho=8, canal=blue

In [13]:
def calcula_no_canal(imagens_no_canal, raio, p_vizinho, doente):
    # Cria uma matriz np.zeros do tamanho da base e com 257 (colunas) sendo o ultimo e o parametros
    # 1 - para imagens saudaveis e 0 - para imagens doentes
    matriz = np.zeros((len(imagens_no_canal), 257))
    matriz.shape
    for i, img in enumerate(imagens_no_canal):
        linha = filtro_LBP(img, raio=raio, p_vizinhos=p_vizinho)
        hist = np.histogram(linha.ravel(), bins=256)[0] # pego apenas a primeira linha do histograma
        matriz[i,:-1] = hist
        matriz[i,-1] = doente
    return matriz

## Armazeno os parametros e executo a função calcula_canal

In [20]:
# parametros a ser passado para a funcao
imagens_no_canal = base_doente_red
raio = 2
p_vizinhos = 4
# doente recebe zero (0) e saudaveis recebe um (1)
doente = 0

# tempo inicial
inicio_paralelo = time.time()
# armazeno a matriz retorna pela funcao
matriz_blp = calcula_no_canal(imagens_no_canal, raio, p_vizinhos, doente)
# tempo de final
fim_paralelo = time.time()
# resultado do processamento
tempo_processamento_paralelo = fim_paralelo-inicio_paralelo
print(tempo_processamento_paralelo)

17745.011971712112


## Salvando no arquivos CSV

In [21]:
np.savetxt('bases/rimone-v2/red_doente200_r2_p4.csv', matriz_blp, fmt="%d", delimiter=",")